In [1]:
%load_ext autoreload

In [2]:
from certified_iris_generator import CertifiedIrisRegionGenerator
import sys
import os
import time
import numpy as np
from functools import partial
import itertools
import mcubes
import visualizations_utils as viz_utils
import iris_utils #TODO remove
from iris_plant_visualizer import IrisPlantVisualizer

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.all import InverseKinematics, RevoluteJoint
import pydrake.symbolic as sym
from pydrake.all import MathematicalProgram
import meshcat

# Build plant

In [4]:

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
parser.package_map().Add( "wsg_50_description", os.path.dirname(FindResourceOrThrow(
            "drake/manipulation/models/wsg_50_description/package.xml")))

simple_collision = True
directives_file = FindResourceOrThrow("drake/sos_iris_certifier/planar_iiwa_simple_collision_welded_gripper.yaml") \
    if simple_collision else FindResourceOrThrow("drake/sos_iris_certifier/planar_iiwa_dense_collision_welded_gripper.yaml")
directives = LoadModelDirectives(directives_file)
models = ProcessModelDirectives(directives, plant, parser)

q0 = [-0.2, -1.2, 1.6]
index = 0
for joint_index in plant.GetJointIndices(models[0].model_instance):
    joint = plant.get_mutable_joint(joint_index)
    if isinstance(joint, RevoluteJoint):
        joint.set_default_angle(q0[index])
        index += 1

plant.Finalize()
# visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, 
#                                       delete_prefix_on_load=False)

# diagram = builder.Build()
# visualizer.load()


# Setup meshcat visualization

In [5]:
do_viz = True
visualizer = IrisPlantVisualizer(plant, builder, scene_graph)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.


# Build Certified Iris Region Object

In [6]:
iris_kwargs = {
    'iris_starting_ellipse_vol': 1e-5,
    'iris_plane_pullback': 1e-4,
    'iris_max_faces': -1
}

In [7]:
iris_generator = CertifiedIrisRegionGenerator(visualizer.diagram, plant, scene_graph, **iris_kwargs)

In [8]:
seed_points = np.tan(np.array([
                        [0.0, -2.016, 1.975], # in tight
                        [-1, -2, 0.5],        # neutral pose
                        [0.3, -0.8, 0.5],     # above shelf
                        [0.25, -1.6, -0.25],  # in shelf 1
                        [0.07, -1.8, -0.2],   # leaving shelf 1
                        [-0.1, -2, -0.3]      # out of shelf 1
                        ])    
                        /2)

regions, ellipses = iris_generator.iris_in_rational_space(seed_points)


0
snopt_example=[ 0.35529068 -0.22984731  0.02559963], growth = 0.05990553086765089
snopt_example=[ 0.28673366 -0.30128467  0.33402563], growth = 0.09485392273531586
snopt_example=[ 0.28776043 -0.30045108  0.32725976], growth = 0.09339195804243797
terminating because a required containment point would have not been contained
Time:   1.67 	Volume:  21.77 	Center: [1.44312073e-09 1.44280166e-09 1.44440504e-09]
0
snopt_example=[ 0.35529266 -0.22984424  0.02559962], growth = 0.059905530783997234
snopt_example=[ 0.30804713 -0.27858144  0.24207296], growth = 0.07703332294756865
terminating because a required containment point would have not been contained
Time:   1.01 	Volume:  21.77 	Center: [1.44312073e-09 1.44280166e-09 1.44440504e-09]
snopt_example=[ 0.30547805 -0.52068241  0.30020753], growth = 354169277.01944494
snopt_example=[ 0.19754042 -0.45436384  0.26846797], growth = 33224401.49490118
snopt_example=[ 0.19598983 -0.45413506  0.27106354], growth = 32414169.41665411
0
snopt_example=

In [9]:
trajectory_start = np.tan(np.array([-1, -2, 0.5])/2) # seed_points[1,:]
trajectory_end = np.tan(np.array([0.25, -1.6, -0.25])/2)  #seed_points[3,:]

In [10]:
# plot regions and collision constraint
if do_viz:
    visualizer.plot_regions(regions, ellipses)
    visualizer.plot_seedpoints(seed_points)
    visualizer.visualize_collision_constraint(N = 50)
    

## Certify the regions

In [11]:
iris_generator.initalize_certifier(plane_order = 1, strict_pos_tol = 1e-4)

Time to initialize region program: 1.0142
time to create Region Certification Problem: : 0.5986
Time to initialize region program: 1.0348
time to create Region Certification Problem: : 0.6304
Time to initialize region program: 1.2945
time to create Region Certification Problem: : 1.4097
Time to initialize region program: 1.4544
time to create Region Certification Problem: : 2.2925
Time to initialize region program: 1.1328
time to create Region Certification Problem: : 1.0335
Time to initialize region program: 1.0962
time to create Region Certification Problem: : 0.8584


{<pydrake.geometry.optimization.HPolyhedron at 0x7f5cc3ef52f0>: <certified_iris_generator.RegionCertificationProblem at 0x7f5d04601790>,
 <pydrake.geometry.optimization.HPolyhedron at 0x7f5cc3ef5230>: <certified_iris_generator.RegionCertificationProblem at 0x7f5d04601df0>,
 <pydrake.geometry.optimization.HPolyhedron at 0x7f5cc3f0adf0>: <certified_iris_generator.RegionCertificationProblem at 0x7f5cc3e91850>,
 <pydrake.geometry.optimization.HPolyhedron at 0x7f5cc3f0adb0>: <certified_iris_generator.RegionCertificationProblem at 0x7f5cc3e91160>,
 <pydrake.geometry.optimization.HPolyhedron at 0x7f5cc3ef04f0>: <certified_iris_generator.RegionCertificationProblem at 0x7f5cc3e91220>,
 <pydrake.geometry.optimization.HPolyhedron at 0x7f5cc3f0ac70>: <certified_iris_generator.RegionCertificationProblem at 0x7f5cc3e91190>}

In [ ]:
do_linesearch_cert = True
if do_linesearch_cert:
    iris_generator.certify_and_adjust_regions_by_linesearch(1e-5)

Starting Region 1/6
min_eps = [0. 0. 0. 0. 0. 0.]
cur_eps = [0.8660254  1.65860451 0.10832327 0.8660254  0.07344629 1.62372753]
max_eps = [1.73205081 3.31720903 0.21664655 1.73205081 0.14689259 3.24745507]
Biggest difference: 3.317209025457233
Smallest difference: 0.14689258966773977
Fixed eps is not success

min_eps = [0.8660254  1.65860451 0.10832327 0.8660254  0.07344629 1.62372753]
cur_eps = [1.29903811 2.48790677 0.16248491 1.29903811 0.11016944 2.4355913 ]
max_eps = [1.73205081 3.31720903 0.21664655 1.73205081 0.14689259 3.24745507]
Biggest difference: 1.6586045127286164
Smallest difference: 0.07344629483386989
Fixed eps is success
Fixed mults is success

min_eps = [0. 0. 0. 0. 0. 0.]
cur_eps = [0.62531182 1.24395338 0.08124244 0.47042867 0.05508471 1.21779565]
max_eps = [1.25062364 2.48790677 0.16248489 0.94085734 0.11016943 2.4355913 ]
Biggest difference: 2.4879067671417374
Smallest difference: 0.1101694254749221
Fixed eps is not success

min_eps = [0.62531182 1.24395338 0.0812

Fixed eps is not success

min_eps = [0.97993278 2.4347021  0.15901009 0.29466014 0.10781342 2.38350542]
cur_eps = [0.98771003 2.45402514 0.16027208 0.29699871 0.10866908 2.40242213]
max_eps = [0.99548727 2.47334817 0.16153406 0.29933729 0.10952474 2.42133884]
Biggest difference: 0.03864606514963276
Smallest difference: 0.0017113240693067272
Fixed eps is not success

min_eps = [0.98771003 2.45402514 0.16027208 0.29699871 0.10866908 2.40242213]
cur_eps = [0.99159865 2.46368665 0.16090307 0.298168   0.10909691 2.41188048]
max_eps = [0.99548727 2.47334817 0.16153406 0.29933729 0.10952474 2.42133884]
Biggest difference: 0.019323032574816157
Smallest difference: 0.0008556620346533567
Fixed eps is not success

min_eps = [0.99159865 2.46368665 0.16090307 0.298168   0.10909691 2.41188048]
cur_eps = [0.99354296 2.46851741 0.16121857 0.29875264 0.10931082 2.41660966]
max_eps = [0.99548727 2.47334817 0.16153406 0.29933729 0.10952474 2.42133884]
Biggest difference: 0.0096615162874083
Smallest diffe

In [ ]:
# uncomment to visualize planes (TODO)
# visualizer.region_to_collision_pair_to_plane_dictionary = iris_generator.linesearch_regions_to_certificates_by_collision_pair_map

In [ ]:
do_alt_cert = True
if do_alt_cert:
    iris_generator.certify_and_adjust_regions_by_bilinear_alternation(max_iter = 1000, 
                                                                 stopped_improving_count_threshold = 5,
                                                                 not_improving_threshold = 1e-4)

Starting Region 1/6
iter 0/1000
Fixed eps is success = True
Fixed mult program is success = True
Cur eps is 
[1.73205081 3.31720902 0.21664655 1.73205081 0.14689259 3.1431506 ]
Improvement 0.10430446575093426
iter 10/1000
Fixed eps is success = True
Fixed mult program is success = True
Cur eps is 
[0.48633229 3.30730932 0.20795123 1.03303765 0.14689259 2.63986438]
Improvement 0.43900804206224525
iter 20/1000
Fixed eps is success = True
Fixed mult program is success = True
Cur eps is 
[0.08045306 3.11130146 0.0700995  0.06110966 0.06706458 2.58305542]
Improvement 0.021629701933557725
iter 30/1000
Fixed eps is success = True
Fixed mult program is success = True
Cur eps is 
[0.05298743 2.95825861 0.0471664  0.03999263 0.04759051 2.52126601]
Improvement 0.015292494006629515
iter 40/1000
Fixed eps is success = True
Fixed mult program is success = True
Cur eps is 
[0.03346805 2.83525892 0.03018183 0.02517927 0.03150892 2.50156269]
Improvement 0.011051636160270646
iter 50/1000
Fixed eps is su

In [ ]:
# plot new regions
if do_viz:
    if do_alt_cert:
        visualizer.plot_regions(iris_generator.alternation_search_regions, region_suffix = '_new_by_alternation')
    if do_linesearch_cert:
        visualizer.plot_regions(iris_generator.linesearch_regions, region_suffix = '_new_by_linesearch')
    

In [ ]:
from pydrake.all import BsplineTrajectoryThroughUnionOfHPolyhedra
# Solve path planning
start_time = time.time()
# trajectory through linesearch_regions
spp = BsplineTrajectoryThroughUnionOfHPolyhedra(trajectory_start.squeeze(), trajectory_end.squeeze(), 
                                                iris_generator.linesearch_regions)
spp.set_max_velocity(0.3*np.ones(plant.num_positions()))
spp.set_extra_control_points_per_region(3)
traj_iris = spp.SolveVerbose()
visualizer.draw_traj_tspace(traj_iris, 100, 'iris')

In [ ]:
print(time.time() - start_time)
print(traj_iris.start_time())
print(traj_iris.end_time())
# draw_traj_tspace(traj_iris, 100, 'iris')
#animate Iris + spp sol
substeps = 1000
its = 10
visualizer.animate_t(traj_iris, substeps, its*substeps)